In [25]:
# прогонка
import math
import numpy as np

a = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]])
b = np.array([0., 0., 1.])
 
print("Обусловленность матрицы a: ", np.linalg.cond(a))
 
# Вывод матрицы на экран
def print_arr( string, namevec, a ):
    if (type(a) == int) or (type(a) == float):
        print(a)
    else:
        print( string )
        for k in range(len(a)):   
            print("{}[{}] = {:8.4f}".format(namevec, k, a[k]))
            
 
# Проверка 3х-диаг. матрицы коэффициентов на корректность
def isCorrectArray(a,b):
    n = len(a)
    
    if np.linalg.matrix_rank(a) != np.linalg.matrix_rank(np.vstack((a.transpose(),b))):
        print('Система несовместна')
        return False
        
    for row in range(1, n - 1):
        if(abs(a[row][row]) < abs(a[row][row - 1]) + abs(a[row][row + 1])):
            print('Не выполнены условия достаточности')
            return False
 
    if (abs(a[0][0]) < abs(a[0][1]))or(abs(a[n - 1][n - 1]) < abs(a[n - 1][n - 2])):
        print('Не выполнены условия достаточности')
        return False
        
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
 
 

def solution(a, b): 
 
    n = len(a)
    x = [0 for k in range(0, n)] # обнуление вектора решений
    print('Размерность матрицы: ',n,'x',n)
    
    # Прямой ход
    v = [0 for k in range(0, n)]
    u = [0 for k in range(0, n)]
    # для первой 0-й строки
    v[0] = a[0][1] / (-a[0][0]) 
    u[0] = ( - b[0]) / (-a[0][0]) 
    for i in range(1, n - 1): # заполняем за исключением 1-й и (n-1)-й строк матрицы
        v[i] = a[i][i+1] / ( -a[i][i] - a[i][i-1]*v[i-1] )
        u[i] = ( a[i][i-1]*u[i-1] - b[i] ) / ( -a[i][i] - a[i][i-1]*v[i-1] )
    # для последней (n-1)-й строки
    v[n-1] = 0
    u[n-1] = (a[n-1][n-2]*u[n-2] - b[n-1]) / (-a[n-1][n-1] - a[n-1][n-2]*v[n-2])
    
    print_arr('Прогоночные коэффициенты v: ','v', v)
    print_arr('Прогоночные коэффициенты u: ','u', u)
    
    # Обратный ход
    x[n-1] = u[n-1]
    for i in range(n-1, 0, -1):
        x[i-1] = v[i-1] * x[i] + u[i-1]
        
    return x    
                
    
if( isCorrectArray(a,b) ):
    x = solution(a, b)  
    print_arr('Решение: ','x', x)

Обусловленность матрицы a:  16.393731622284385
Не выполнены условия достаточности


In [34]:
# якоби
import numpy as np
import math
import copy

a = np.array([[2,1,0,0],[1,10,-5,0],[0,1,-5,2],[0,0,1,4]])
 
b = np.array([-5,-18,-40,-27])


print("Обусловленность матрицы А: ", np.linalg.cond(a))

def print_arr( string, namevec, a ):
    if (type(a) == int) or (type(a) == float):
        print(a)
    else:
        print( string )
        for k in range(len(a)):   
            print("{}[{}] = {:8.4f}".format(namevec, k, a[k]))
            

 
# Проверка матрицы коэффициентов на корректность
def isCorrectArray(a):
    if np.linalg.matrix_rank(a) != np.linalg.matrix_rank(np.vstack((a.transpose(),b))):
        print('Система несовместна')
        return False
    for r in range(0, len(a)):
        s = 0
        for row in range(0, len(a)):
            s=s+a[r][row]
        if abs(a[r][r]) <= abs(s-a[r][r]):
            print('Не выполняется диагональное преобладание')
            return False
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
# Условие завершения программы на основе вычисления
# расстояния между соответствующими элементами соседних
# итераций в методе решения 
def isNeedToComplete(x_old, x_new):
    eps = 0.0001
    sum_up = 0
    sum_low = 0
    for k in range(0, len(x_old)):
        sum_up += ( x_new[k] - x_old[k] ) ** 2
        sum_low += ( x_new[k] ) ** 2
        
    return math.sqrt( sum_up / sum_low ) < eps
 
# Процедура решения
def solution(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b) # количество корней
        
        x = [1 for k in range(0, count) ] # начальное приближение корней
        
        numberOfIter = 0  # подсчет количества итераций
        MAX_ITER = 100    # максимально допустимое число итераций
        while( numberOfIter < MAX_ITER ):
 
            x_prev = copy.deepcopy(x)
            y = np.zeros(count+1)
            for i in range(0, count):y[i] = x[i]
            
            for k in range(0, count):
                S = b[k]
                for j in range(0, count):
                    if( j != k ): S = S - a[k][j] * y[j] 
                x[k] = S / a[k][k]
            
            if isNeedToComplete(x_prev, x) : # проверка на выход
                break
              
            numberOfIter += 1
 
        print('Количество итераций на решение: ', numberOfIter)
        
        return x    
                
    
print_arr('Решение: ','x', solution(a, b))

Обусловленность матрицы А:  6.414470659550419
Количество итераций на решение:  9
Решение: 
x[0] =  -2.9999
x[1] =   1.0001
x[2] =   5.0000
x[3] =  -8.0000


In [30]:
# зейделя
import numpy as np
import math
import copy

a = np.array([[2,1,0,0],[1,10,-5,0],[0,1,-5,2],[0,0,1,4]])
 
b = np.array([-5,-18,-40,-27])


print("Обусловленность матрицы А: ", np.linalg.cond(a))

def print_arr( string, namevec, a ):
    if (type(a) == int) or (type(a) == float):
        print(a)
    else:
        print( string )
        for k in range(len(a)):   
            print("{}[{}] = {:8.4f}".format(namevec, k, a[k]))

 
# Проверка матрицы коэффициентов на корректность
def isCorrectArray(a):
    if np.linalg.matrix_rank(a) != np.linalg.matrix_rank(np.vstack((a.transpose(),b))):
        print('Система несовместна')
        return False
    if np.all(np.linalg.eigvals(a) <= 0):
        print('Матрица не положительно определенная')
        return False
    if np.all(a.transpose() != a):
        print('Матрица не симметричная')
        return False
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
# Условие завершения программы на основе вычисления
# расстояния между соответствующими элементами соседних
# итераций в методе решения 
def isNeedToComplete(x_old, x_new):
    eps = 0.0001
    sum_up = 0
    sum_low = 0
    for k in range(0, len(x_old)):
        sum_up += ( x_new[k] - x_old[k] ) ** 2
        sum_low += ( x_new[k] ) ** 2
        
    return math.sqrt( sum_up / sum_low ) < eps
 
# Процедура решения
def solution(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b) # количество корней
        
        x = [1 for k in range(0, count) ] # начальное приближение корней
        
        numberOfIter = 0  # подсчет количества итераций
        MAX_ITER = 100    # максимально допустимое число итераций
        while( numberOfIter < MAX_ITER ):
 
            x_prev = copy.deepcopy(x)

            for k in range(0, count):
                S = b[k]
                for j in range(0, count):
                    if( j != k ): S = S - a[k][j] * x[j] 
                x[k] = S / a[k][k]
                        
            
            if isNeedToComplete(x_prev, x) : # проверка на выход
                break
              
            numberOfIter += 1
 
        print('Количество итераций на решение: ', numberOfIter)
        
        return x    
                
    
print_arr('Решение: ','x', solution(a, b))

Обусловленность матрицы А:  6.414470659550419
Количество итераций на решение:  6
Решение: 
x[0] =  -3.0000
x[1] =   1.0000
x[2] =   5.0000
x[3] =  -8.0000


In [3]:
# минимальных невязок
import numpy as np


def print_arr( string, namevec, a ):
    if (type(a) == int) or (type(a) == float):
        print(a)
    else:
        print( string )
        for k in range(len(a)):   
            print("{}[{}] = {:8.4f}".format(namevec, k, a[k]))

def isCorrectArray(a):
    if np.linalg.matrix_rank(a) != np.linalg.matrix_rank(np.vstack((a.transpose(),b))):
        print('Система несовместна')
        return False
    
    if np.all(np.linalg.eigvals(a) <= 0):
        print('Матрица не положительно определенная')
        return False
    if np.all(a.transpose() != a):
        print('Матрица не симметричная')
        return False
    return True 

def solution(a, b):
    N=0
    x = b
    eps=0.01
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
    else:
        while (np.linalg.norm(a@x - b)>eps):
            N+=1
            r = a@x - b  # высчитываем направление невязки (вектор)
            # высчитываем коэф. поправки вектора невязки
            t = np.dot((a * r).transpose(), r) / (np.linalg.norm(a * r)**2)
            
            x = x - t * r  # новое решение СЛАУ
            
        print_arr('Решение: ','x', x)
        print('Количество итераций на решение: ', N) 

a = np.array([[2,1,0,0],[1,10,-5,0],[0,1,-5,2],[0,0,1,4]])
 
b = np.array([-5,-18,-40,-27])

print("Обусловленность матрицы А: ", np.linalg.cond(a)) 

solution(a, b) 


Обусловленность матрицы А:  6.414470659550419
Решение: 
x[0] =  -3.0030
x[1] =   0.9994
x[2] =   4.9987
x[3] =  -8.0008
Количество итераций на решение:  99


In [66]:
#наискорейшего спуска
import numpy as np

print("Обусловленность матрицы А: ", np.linalg.cond(a)) 

def print_arr( string, namevec, a ):
    if (type(a) == int) or (type(a) == float):
        print(a)
    else:
        print( string )
        for k in range(len(a)):   
            print("{}[{}] = {:8.4f}".format(namevec, k, a[k]))

def isCorrectArray(a):
    if np.linalg.matrix_rank(a) != np.linalg.matrix_rank(np.vstack((a.transpose(),b))):
        print('Система несовместна')
        return False
    
    if np.all(np.linalg.eigvals(a) <= 0):
        print('Матрица не положительно определенная')
        return False
    if np.all(a.transpose() != a):
        print('Матрица не симметричная')
        return False
    return True 

def solution(a, b):
    N=0
    x = b
    eps=0.01
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
    else:
        while (np.linalg.norm(a@x - b)>eps):
            N+=1
            r = a@x - b  # высчитываем направление невязки (вектор)
            # высчитываем коэф. поправки вектора невязки
        
            t = (np.linalg.norm(r)**2) / np.dot((a * r), r)
        
            x = x - t*r  # новое решение СЛАУ
            
            
        print_arr('Решение: ','x', x)
        print('Количество итераций на решение: ', N) 

a = np.array([[1,1],[1,2]])
b = np.array([1,1])


solution(a, b) 



Обусловленность матрицы А:  6.414470659550419
Решение: 
x[0] =   0.9971
x[1] =  -0.0025
Количество итераций на решение:  39


In [11]:
import numpy as np

a = np.array([[4,-3,2,-1],[3,-2,1,-3],[5,-3,1,-8]])
 
b = np.array([8,7,1])


# --- перемена местами двух строк системы
def SwapRows(A, B, row1, row2): 
    A[row1], A[row2] = A[row2], A[row1] 
    B[row1], B[row2] = B[row2], B[row1]

 # --- деление строки системы на число
def DivideRow(A, B, row, divider):
    A[row] = [a / divider for a in A[row]] 
    B[row] /= divider

# --- сложение строки системы с другой строкой, умноженной на число
def CombineRows(A, B, row, source_row, weight):
    A[row] = [(a + k * weight) for a, k in zip(A[row], A[source_row])]
    B[row] += B[source_row] * weight

# --- приведением к треугольному виду
def Gauss(A, B):
    column = 0
    while (column < len(B)):
        current_row = None
        
        for r in range(column, len(A)):
            if current_row is None or abs(A[r][column]) > abs(A[current_row][column]):
                current_row = r 
        
        if current_row is None:
            return None
        
        if current_row != column: 
            SwapRows(A, B, current_row, column)
            
        DivideRow(A, B, column, A[column][column]) 
        
        for r in range(column + 1, len(A)):
            CombineRows(A, B, r, column, -A[r][column])
        
        column += 1
    print(A)   
    l=len(A)-1
    k=0
    for r in range(0, l+1):
            if A[l][r] != 0:
                return False
    return True           
Gauss(a, b)   

[[ 1  0  0  0]
 [ 0  1  0  1]
 [ 0  0  1 -1]]


False

In [1]:
# -*- coding: utf-8 -*-

import math

import time

from tkinter import *

 

scene_width = 640

scene_height = 480

 

c = Canvas(width=scene_width, height=scene_height, bg='grey80')

c.pack()

cannon_length = 70

v = 70

g = 9.8

 

def rotate_cannon(x_i, y_i):

    mn = y_i/x_i

    y = (mn * cannon_length) / (math.sqrt(mn * mn + 1))

    x = math.sqrt(cannon_length * cannon_length - y * y)

    c.coords(cn, 0, scene_height-10, int(x), scene_height-10-int(y))

    angle = math.atan2(x_i, y_i) # угол наклона ствола пушки

    t = 2*v*math.sin(1.5707963267948966-angle)/g

    x_t = v*t*math.cos(1.5707963267948966-angle) # координата прицела

    c.coords(cursor, int(x_t-5), scene_height-15, int(x_t+5), scene_height-5)

    return

 

def motion(event):

    if event.x > 0 and event.y < scene_height-10:

        rotate_cannon(float(event.x), float(470-event.y))

    return

 

def shoot_ball(event):

    ball = c.create_oval(-5, scene_height-15, 5, scene_height-5, fill='yellow')

    angle = math.atan2(float(event.x), float(470-event.y)) # угол выстрела

    delta_t = 2*v*math.sin(1.5707963267948966-angle)/g # время полета снаряда

    x_t = v*delta_t*math.cos(1.5707963267948966-angle)/10 # расстояние приземления снаряда

    y_max = v*delta_t/2*math.sin(1.5707963267948966-angle)-g*delta_t*delta_t/8 # максимальная высота полета снаряда



    length = 2*math.sqrt(x_t*x_t/4+y_max*y_max) # грубая оценка длины траектории

    num_frames = int(length/delta_t) # количество кадров для отрисовки

    delta_t = delta_t / num_frames

    for i in range(num_frames): # цикл отрисовки

        x_ball = v*(i+1)*delta_t*math.cos(1.5707963267948966-angle)

        y_ball = v*(i+1)*delta_t*math.sin(1.5707963267948966-angle)-g*(i+1)*delta_t*(i+1)*delta_t/2

        c.coords(ball, int(x_ball-5), scene_height-y_ball-15, int(x_ball+5), scene_height-y_ball-5)

        c.update()
    
        time.sleep(0.02)

        c.coords(ball, int(x_ball - 5), scene_height - y_ball - 15, int(x_ball + 5), scene_height - y_ball - 5)

    return

 

c.bind('<Motion>', motion)

c.bind('<Button 1>', shoot_ball)

c.create_oval(-25, 445, 25, 495, fill='#141414')

c.create_rectangle(25, scene_height-10, 0, scene_height, fill='#141414')

cn = c.create_line(0, scene_height-10, cannon_length, scene_height-10, width=10, fill='#141414')

c.create_line(0, scene_height-10, scene_width, scene_height-10, width=1, fill='black')

cursor=c.create_oval(cannon_length, scene_height-15, cannon_length+10, scene_height-5, fill='red')

 

mainloop()